In [59]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from citipy import citipy
import requests
import os
import gmaps
from config import weather_api_key
from config import g_key

In [60]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [61]:
#load weatherpy df
city_data_to_load = os.path.join("weather_data","WeatherPy_database.csv")
city_data_df = pd.read_csv(city_data_to_load)
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Rikitea,PF,-23.12,-134.97,73.74,84,79,12.48,broken clouds,0.00,0.0
1,1,Rocha,UY,-34.48,-54.33,43.00,73,73,6.42,broken clouds,0.00,0.0
2,2,Vegachi,CO,6.76,-74.79,73.89,96,72,1.59,moderate rain,2.95,0.0
3,3,Busselton,AU,-33.65,115.33,57.00,89,11,12.24,few clouds,0.00,0.0
4,4,Port-Gentil,GA,-0.72,8.78,73.40,94,40,5.82,scattered clouds,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
561,561,Gidole,ET,5.65,37.37,58.82,88,90,1.63,overcast clouds,0.00,0.0
562,562,Zhigansk,RU,66.77,123.37,70.95,44,71,7.11,broken clouds,0.00,0.0
563,563,Nipawin,CA,53.37,-104.00,66.20,68,90,3.36,overcast clouds,0.00,0.0
564,564,Nuuk,GL,64.18,-51.72,44.60,75,20,9.17,few clouds,0.00,0.0


In [62]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_question = str(input("Do you want it to be raining? (yes/no) "))
snow_question = str(input("Do you want it to be snowing? (yes/no) "))
# Filter the dataset to find the cities that fit the criteria.

if rain_question =="yes" and snow_question =="yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                           (city_data_df["Max Temp"] >= min_temp) &
                                           (city_data_df["Rain (inches)"] > 0.0) &
                                           (city_data_df["Snow (inches)"] > 0.0)]
    
elif rain_question =="yes" and snow_question =="no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                           (city_data_df["Max Temp"] >= min_temp) &
                                           (city_data_df["Rain (inches)"] > 0.0) &
                                           (city_data_df["Snow (inches)"] == 0.0)]
    
elif rain_question =="no" and snow_question =="yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                           (city_data_df["Max Temp"] >= min_temp) &
                                           (city_data_df["Rain (inches)"] == 0.0) &
                                           (city_data_df["Snow (inches)"] > 0.0)]    
    
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] == 0.0) &
                                          (city_data_df["Snow (inches)"] == 0.0)]
    
    
preferred_cities_df

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 82
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Rikitea,PF,-23.12,-134.97,73.74,84,79,12.48,broken clouds,0.0,0.0
4,4,Port-Gentil,GA,-0.72,8.78,73.40,94,40,5.82,scattered clouds,0.0,0.0
10,10,Atuona,PF,-9.80,-139.03,80.58,71,0,13.31,clear sky,0.0,0.0
13,13,Ribeira Grande,PT,38.52,-28.70,68.00,88,40,11.41,scattered clouds,0.0,0.0
22,22,Bengkulu,ID,-3.80,102.27,75.87,83,99,4.25,overcast clouds,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
549,549,Miyako,JP,39.64,141.95,69.62,86,81,4.00,broken clouds,0.0,0.0
552,552,Stuttgart,DE,48.78,9.18,66.99,93,97,1.12,overcast clouds,0.0,0.0
554,554,Poum,NC,-20.23,164.02,70.84,67,80,12.93,broken clouds,0.0,0.0
562,562,Zhigansk,RU,66.77,123.37,70.95,44,71,7.11,broken clouds,0.0,0.0


In [63]:
# Create DataFrame to store hotel names along with city, country, max temp, description, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
541,Cabedelo,BR,75.20,broken clouds,-6.98,-34.83,
542,Acarau,BR,74.84,overcast clouds,-2.89,-40.12,
543,Guaruja,BR,66.99,moderate rain,-23.99,-46.26,
544,Ibotirama,BR,75.60,scattered clouds,-12.19,-43.22,
546,Salinas,US,78.01,clear sky,36.68,-121.66,
549,Miyako,JP,69.62,broken clouds,39.64,141.95,
552,Stuttgart,DE,66.99,overcast clouds,48.78,9.18,
554,Poum,NC,70.84,broken clouds,-20.23,164.02,
562,Zhigansk,RU,70.95,broken clouds,66.77,123.37,
563,Nipawin,CA,66.20,overcast clouds,53.37,-104.00,


In [64]:
hotel_df.count()

City                   216
Country                216
Max Temp               216
Current Description    216
Lat                    216
Lng                    216
Hotel Name             216
dtype: int64

In [65]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError, KeyError):
        print("No hotel")
    else:
        if hotel_df["Hotel Name"] is None:
            pass

No hotel
No hotel
No hotel
No hotel
No hotel
No hotel
No hotel
No hotel
No hotel
No hotel
No hotel
No hotel
No hotel
No hotel


In [66]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,73.74,broken clouds,-23.12,-134.97,Pension Maro'i
4,Port-Gentil,GA,73.40,scattered clouds,-0.72,8.78,Fengshui Residence
10,Atuona,PF,80.58,clear sky,-9.80,-139.03,Villa Enata
13,Ribeira Grande,PT,68.00,scattered clouds,38.52,-28.70,Quinta da Meia Eira
22,Bengkulu,ID,75.87,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
...,...,...,...,...,...,...,...
549,Miyako,JP,69.62,broken clouds,39.64,141.95,Hotel Route Inn Miyako
552,Stuttgart,DE,66.99,overcast clouds,48.78,9.18,Hotel Motel One Stuttgart-Mitte
554,Poum,NC,70.84,broken clouds,-20.23,164.02,Gîte kajeon
562,Zhigansk,RU,70.95,broken clouds,66.77,123.37,Gostinichnyy Kompleks Aylgy


In [67]:
# Remove rows with blank Hotel(nothing in 5km)
empty = hotel_df["Hotel Name"] != ""
hotel_df = hotel_df[empty]
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,73.74,broken clouds,-23.12,-134.97,Pension Maro'i
4,Port-Gentil,GA,73.40,scattered clouds,-0.72,8.78,Fengshui Residence
10,Atuona,PF,80.58,clear sky,-9.80,-139.03,Villa Enata
13,Ribeira Grande,PT,68.00,scattered clouds,38.52,-28.70,Quinta da Meia Eira
22,Bengkulu,ID,75.87,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
...,...,...,...,...,...,...,...
549,Miyako,JP,69.62,broken clouds,39.64,141.95,Hotel Route Inn Miyako
552,Stuttgart,DE,66.99,overcast clouds,48.78,9.18,Hotel Motel One Stuttgart-Mitte
554,Poum,NC,70.84,broken clouds,-20.23,164.02,Gîte kajeon
562,Zhigansk,RU,70.95,broken clouds,66.77,123.37,Gostinichnyy Kompleks Aylgy


In [68]:
# Create the output file (CSV).
output_data_file2 = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file2, index_label="City_ID")

In [69]:
#create popup box
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F </dd>
</dl>
"""

In [70]:
# Store the DF
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [71]:
# Add a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))